In [0]:
# !pip install --upgrade tables
# !pip install eli5
# !pip install xgboost

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

## Reading data 

In [0]:
# cd "drive/My Drive/Colab Notebooks/dw_matrix/matrix_two"

In [47]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
df = df[ df['price_currency'] != 'EUR' ]

## Feature engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat: 
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [50]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X =  df[ feats ].values
  y = df['price_value'].values


  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree

In [52]:
model = DecisionTreeRegressor(max_depth=5)
run_model(model, cat_feats)

(-19566.58893736832, 90.61814865166077)

## RandomForest

In [53]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18734.2072708522, 109.87074106274046)

## XGBoost

In [55]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[14:28:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:29:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:29:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13039.290196724838, 109.36715375706265)

In [56]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X,y)

imp = PermutationImportance(m).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[14:30:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1261 ± 0.0025,param_napęd__cat
0.1149 ± 0.0025,param_stan__cat
0.1124 ± 0.0019,param_rok-produkcji__cat
0.0568 ± 0.0023,param_faktura-vat__cat
0.0478 ± 0.0013,param_moc__cat
0.0408 ± 0.0008,param_skrzynia-biegów__cat
0.0257 ± 0.0005,param_marka-pojazdu__cat
0.0248 ± 0.0012,feature_kamera-cofania__cat
0.0227 ± 0.0005,param_typ__cat
0.0197 ± 0.0014,param_pojemność-skokowa__cat


In [57]:
feats = ['param_napęd__cat',
'param_faktura-vat__cat',
'feature_kamera-cofania__cat',
'param_rok-produkcji__cat',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_moc__cat',
'feature_regulowane-zawieszenie__cat',
'seller_name__cat',
'param_pojemność-skokowa__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_światła-led__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_bluetooth__cat',
'param_marka-pojazdu__cat',
'feature_isofix__cat',
'feature_elektryczne-szyby-przednie__cat']

len(feats)

20

In [58]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[14:36:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:36:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:36:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13835.171790685052, 123.24356958064017)

In [59]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1959', '1990', '1991', '1974', None, '1975', '1973', '1985',
       '1984', '1986', '1981', '1979', '1960', '1983', '1978', '1964',
       '1980', '1972', '1969', '1956', '1966', '1977', '1971', '1963',
       '1953', '1961', '1952', '1949', '1976', '1965', '1937', '1968',
       '1958', '1962', '1955', '1970', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [60]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))

feats = ['param_napęd__cat',
'param_faktura-vat__cat',
'feature_kamera-cofania__cat',
'param_rok-produkcji',    #numerical
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_moc__cat',
'feature_regulowane-zawieszenie__cat',
'seller_name__cat',
'param_pojemność-skokowa__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_światła-led__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_bluetooth__cat',
'param_marka-pojazdu__cat',
'feature_isofix__cat',
'feature_elektryczne-szyby-przednie__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[14:37:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:37:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:37:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11972.1866659862, 66.60594143161778)

In [64]:
df['param_moc'] = df['param_moc'].map( lambda x: -1 if str(x)=='None' else int(str(x).split(' ')[0]) )


feats = ['param_napęd__cat',
'param_faktura-vat__cat',
'feature_kamera-cofania__cat',
'param_rok-produkcji',
'param_stan__cat',      #numerical
'param_skrzynia-biegów__cat',
'param_moc',            #numerical
'feature_regulowane-zawieszenie__cat',
'seller_name__cat',
'param_pojemność-skokowa__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_światła-led__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_bluetooth__cat',
'param_marka-pojazdu__cat',
'feature_isofix__cat',
'feature_elektryczne-szyby-przednie__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[14:40:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:40:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:40:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10046.038458664274, 77.90273079566495)

In [65]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map( lambda x: -1 if str(x)=='None' else int(str(x).split('cm')[0].replace(' ','')) )


feats = ['param_napęd__cat',
'param_faktura-vat__cat',
'feature_kamera-cofania__cat',
'param_rok-produkcji',       #numerical
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_moc',                 #numerical
'feature_regulowane-zawieszenie__cat',
'seller_name__cat',
'param_pojemność-skokowa',   #numerical
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_światła-led__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_bluetooth__cat',
'param_marka-pojazdu__cat',
'feature_isofix__cat',
'feature_elektryczne-szyby-przednie__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[14:41:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:41:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:41:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9891.837420462847, 70.25127208891577)